In [14]:
import numpy as np
import math
from typing import Callable, Union, Tuple, List, Any


# Initial-Value Problems for Ordinary Differential Equations

## Chapter 5.2 Euler's Method
Given 
$$ \frac{dy}{dt} = f(t, y),\,\,\, a\leq t\leq b,\,\,\,y(a)=\alpha $$
and consider
$$ t_i = a+ih,\,\,\,i=0,1,2\cdots,N$$
Suppose $y(t)$ has two continuous derivatives on $[a, b]$, then by Taylor's Theorem
$$ y(t_{i+1})=y(t_i)+(t_{i+1}-t_i)y'(t_i)+\frac{(t_{i+1}-t_i)^2}{2}y''(\xi_i) $$
because $h=t_{i+1}-t_i$, then
$$ y(t_{i+1})=y(t_i)+hf(t_i,y(t_i))+\frac{h^2}{2}y''(\xi_i)$$

The definition of $t_i$ and $h$ are the same throughout the context.

Euler's method constructs $w_i\approx y(t_i)$, by deleting the remainder term. In short,
$$w_0=\alpha$$
$$w_{i+1}=w_i+hf(t_i, w_i),\,\,\,i=0, 1,\cdots,N-1$$

In [15]:
def Euler(f:Callable[[Union[float, int], Union[float, int]], Union[float, int]],
          a:Union[float, int],
          b:Union[float, int],
          alpha:Union[float, int],
          h:Union[float, int]=None,
          N:int=None,
         ) -> Tuple[np.ndarray[Union[float, int], Any], np.ndarray[Union[float, int], Any]]:
    if (h is None) and (N is None):
        raise ValueError("h and N cannot be None at the same time")
    elif h is None:
        h = (b - a) / N
    elif N is None:
        N = int((b - a) / h)
    t_, w_ = np.linspace(a, b, N + 1), np.ones((N + 1, )) * alpha
    for idx in range(1, N + 1):
        w_[idx] =  w_[idx - 1] + h * f(t_[idx - 1], w_[idx - 1])
    return (t_, w_)

In [16]:
f = lambda t, y: y - t ** 2 + 1
Euler(f, 0, 2, 0.5, h=0.5)

(array([0. , 0.5, 1. , 1.5, 2. ]),
 array([0.5   , 1.25  , 2.25  , 3.375 , 4.4375]))

In [17]:
f = lambda t, y: np.cos(2 * t) + np.sin(3 * t)
Euler(f, 0, 1, 1, N=4)

(array([0.  , 0.25, 0.5 , 0.75, 1.  ]),
 array([1.        , 1.25      , 1.63980533, 2.02425465, 2.23645725]))

## Chapter 5.3 Higher-Order Taylor Methods
Suppose the solution $y(t)$ to the initial-value problem
$$ \frac{dy}{dt} = f(t, y),\,\,\, a\leq t\leq b,\,\,\,y(a)=\alpha $$
has $(n+1)$ continuous derivatives. By its $n$th Taylor polynomial about $t_i$ and evaluate at $t_{i+1}$,
$$y(t_{{i+1}})=y(t_i)+hy'(t_i)+\frac{h^2}{2}y''(t_i)+\cdots+\frac{h^n}{n!}y^{(n)}(t_i)+\frac{h^{n+1}}{(n+1)!}y^{(n+1)}(\xi_i)$$
for some $\xi_i\in(t_i, t_{i+1})$
Successive differentiation of the solution, $y(t)$ gives
$$y'(t)=f(t, y(t)),\,\,\,y''(t)=f'(t, y(t)),\,\,\,\text{ and generally, }\,\,\, y^{(k)}(t)=f^{(k-1)}(t, y(t))$$
Substituting these findings into previous equations gives
$$y(t_{{i+1}})=y(t_i)+hf(t_i)+\frac{h^2}{2}f'(t_i)+\cdots+\frac{h^n}{n!}f^{(n - 1)}(t_i)+\frac{h^{n+1}}{(n+1)!}f^{(n)}(\xi_i)$$
Similar to Euler's method, in short
$$w_0=\alpha$$
$$w_{i+1}=w_i+hT^{(n)}(t_i, w_i),\,\,\,i=0,1,\cdots,N-1$$
where
$$T^{(n)}(t_i, w_i)=\sum^{n-1}_{i=0}\frac{h^{i}}{(i+1)!}f^{(i)}(t_i, w_i)$$
Euler's method is Taylor's method of order one.

In [18]:
def Taylor(func:List[Callable[[Union[float, int], Union[float, int]], Union[float, int]]],
           a:Union[float, int],
           b:Union[float, int],
           alpha:Union[float, int],
           h:Union[float, int]=None,
           N:int=None
          ) -> Tuple[np.ndarray[Union[float, int], Any], np.ndarray[Union[float, int], Any]]:
    if (h is None) and (N is None):
        raise ValueError("h and N cannot be None at the same time")
    elif h is None:
        h = (b - a) / N
    elif N is None:
        N = int((b - a) / h)
    t_, w_ = np.linspace(a, b, N + 1), np.ones((N + 1, )) * alpha

    # factorial = [1] * (len(func) + 1)
    # for idx in range(2, len(func) + 1):
    #     factorial[idx] = factorial[idx - 1] * (idx)
    
    for idx in range(1, N + 1):
        T = sum([(h ** jdx) * f(t_[idx - 1], w_[idx - 1]) / math.factorial(jdx + 1) for jdx, f in enumerate(func)])
        w_[idx] = w_[idx - 1] + h * T
    return (t_, w_)

In [19]:
f = [
    lambda t, y: y - t ** 2 + 1,
    lambda t, y: y - t ** 2 + 1 - 2 * t
]
Taylor(f, 0, 2, 0.5, 0.2)

(array([0. , 0.2, 0.4, 0.6, 0.8, 1. , 1.2, 1.4, 1.6, 1.8, 2. ]),
 array([0.5       , 0.83      , 1.2158    , 1.652076  , 2.13233272,
        2.64864592, 3.19134802, 3.74864458, 4.30614639, 4.8462986 ,
        5.34768429]))

## Chapter 5.4 Runge-Kutta Methods
__Runge-Kutta methods__ have the high-order local truncation error of the Taylor methods
but eliminate the need to compute and evaluate the derivatives of $f (t, y)$

In general, __Runge-Kutta Methods__ have the following form:
$$
y_{i+1} = y_i+h\sum_{i=1}^{s}b_ik_i
$$
where
$y_i=y(t_i)$

$k_1=f(t_i,y_i)$

$k_2=f(t_i+c_2h,y_i+(a_{21}k_1)h)$

$k_3=f(t_i+c_3h,y_i+(a_{31}k_1+a_{32}k_2)h)$

$\vdots$

$k_s=f(t_i+c_sh,y_i+(a_{s1}k_1+a_{s2}k_2+\cdots+a_{s, s-1}k_{s-1})h)$

The parameters above can be store as an matrix where the illustrated as below:
\begin{bmatrix}
0 & 0 & 0 & \cdots & 0\\
c_2 & a_{21} & a_{22} & \cdots & a_{s2}\\
\vdots & \vdots & \vdots & \ddots & \vdots\\
c_s & a_{s1} & a_{s2} & \cdots & a_{ss}\\
0 & b_1 & b_2 & \cdots & b_s
\end{bmatrix}	

The matrix $[a_{ij}]$ is called the Runge-Kutta matrix while $b_i$ and $c_i$ are known as the weights and the nodes.

Refer to [Wikipedia](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods) for more information

### Runge-Kutta Methods of Order Two
There is a family of such methods, parameterized by α and given by the formula
$$
y(t_{i+1})=y(t_i)+h\biggl(\biggl(1-\frac{1}{2\alpha}f(t_i, y(t_i))\biggr)+\frac{1}{2\alpha}f(t_i+\alpha h, y(t_i)+\alpha f(t_i, y(t_i)))\biggr)
$$
where 
$\alpha=\frac{1}{2}$ gives the midpoint method, $\alpha=1$ is Modified Euler Method(Heun's method), and $\alpha=\frac{2}{3}$ is Ralston's method.

So by the matrix notation introduced above, we have
\begin{bmatrix}
0 & 0 & 0\\
\alpha & \alpha & 0\\
0 & 1 - \frac{1}{2\alpha} & \frac{1}{2\alpha}
\end{bmatrix}	


#### Midpoint Method
$w_0=\alpha$

$w_{i+1}=w_i+hf\biggl(t_i+\frac{h}{2}, w_i+\frac{h}{2}f(t_i, w_i)\biggr),\,\,\,i=0, 1,\cdots,N-1$

#### Modified Euler Method
$w_0=\alpha$

$w_{i+1}=w_i+\frac{h}{2}[f(t_i, w_i)+f(t_{i+1}, w_i+hf(t_i, w_i))],\,\,\,i=0, 1,\cdots,N-1$

#### Ralston's Method
$w_0=\alpha$

$w_{i+1}=w_i+h\biggl[\frac{1}{4}f(t_i, w_i)+\frac{3}{4}f(t_{i+1}+\frac{2}{3}h, w_i+\frac{2}{3}hf(t_i, w_i))\biggr],\,\,\,i=0, 1,\cdots,N-1$

In [20]:
def RK2_Generic(f:Callable[[Union[float, int], Union[float, int]], Union[float, int]],
          a:Union[float, int],
          b:Union[float, int],
          y_init:Union[float, int],
          h:Union[float, int]=None,
          alpha:float=0.5,
          N:int=None,
         ) -> Tuple[np.ndarray[Union[float, int], Any], np.ndarray[Union[float, int], Any]]:
    if (h is None) and (N is None):
        raise ValueError("h and N cannot be None at the same time")
    elif h is None:
        h = (b - a) / N
    elif N is None:
        N = int((b - a) / h)
    t_, w_ = np.linspace(a, b, N + 1), np.ones((N + 1, )) * y_init
    for idx in range(1, N + 1):
        w_[idx] =  w_[idx - 1] + h * ((1 - 1 / (2 * alpha)) * f(t_[idx - 1], w_[idx - 1]) + 
                                      (f(t_[idx - 1] + alpha * h, w_[idx - 1] + alpha * h * f(t_[idx - 1], w_[idx - 1]))) / (2 * alpha)
                                      )
    return (t_, w_)

### Runge-Kutta Methods of Order Three
#### Heun's Method

$w_0=\alpha$

$w_{i+1}=w_i+\frac{h}{4}\biggl[f(t_i, w_i)+3f(t_{i}+\frac{2}{3}h, w_i+\frac{2}{3}hf(t_i+\frac{1}{3}h, w_i+\frac{1}{3}f(t_i, w_i)))\biggr],\,\,\,i=0, 1,\cdots,N-1$

In [21]:
def Heun(f:Callable[[Union[float, int], Union[float, int]], Union[float, int]],
          a:Union[float, int],
          b:Union[float, int],
          y_init:Union[float, int],
          h:Union[float, int]=None,
          N:int=None,
         ) -> Tuple[np.ndarray[Union[float, int], Any], np.ndarray[Union[float, int], Any]]:
    if (h is None) and (N is None):
        raise ValueError("h and N cannot be None at the same time")
    elif h is None:
        h = (b - a) / N
    elif N is None:
        N = int((b - a) / h)
    t_, w_ = np.linspace(a, b, N + 1), np.ones((N + 1, )) * y_init
    for idx in range(1, N + 1):
        w_[idx] =  w_[idx - 1] + h * (f(t_[idx - 1], w_[idx - 1]) + 3 * f(t_[idx - 1] + (2 / 3) * h, w_[idx - 1] + (2 / 3) * 
                                        h * f(t_[idx - 1] + (1 / 3) * h, w_[idx - 1] + (1 / 3) * h *  f(t_[idx - 1], w_[idx - 1])))) / 4
    return (t_, w_)

### Runge-Kutta Methods of Order Four (Most commonly used)

$w_0=\alpha$

$k_1=hf(t_i, w_i)$

$k_2=hf\biggl(t_i+\frac{h}{2}, w_i+\frac{1}{2}k_1\biggr)$

$k_3=hf\biggl(t_i+\frac{h}{2}, w_i+\frac{1}{2}k_2\biggr)$

$k_4=hf(t_{i+1}, w_i+k_3)$

$w_{i+1}=\frac{1}{6}(k_1+2k_2+2k_3+k_4)$

for each $i=0, 1,\cdots,N-1$

In [22]:
def RK4(f:Callable[[Union[float, int], Union[float, int]], Union[float, int]],
        a:Union[float, int],
        b:Union[float, int],
        y_init:Union[float, int],
        h:Union[float, int]=None,
        N:int=None,
        ) -> Tuple[np.ndarray[Union[float, int], Any], np.ndarray[Union[float, int], Any]]:
    if (h is None) and (N is None):
        raise ValueError("h and N cannot be None at the same time")
    elif h is None:
        h = (b - a) / N
    elif N is None:
        N = int((b - a) / h)
    t_, w_ = np.linspace(a, b, N + 1), np.ones((N + 1, )) * y_init
    for idx in range(1, N + 1):
        k_1 = h * f(t_[idx - 1], w_[idx - 1])
        k_2 = h * f(t_[idx - 1] + h / 2, w_[idx - 1] + k_1 / 2)
        k_3 = h * f(t_[idx - 1] + h / 2, w_[idx - 1] + k_2 / 2)
        k_4 = h * f(t_[idx], w_[idx - 1] + k_3)
        w_[idx] = w_[idx - 1] + (k_1 + 2 * k_2 + 2 * k_3 + k_4) / 6
    return (t_, w_)

#### Implementation of General Runge-Kutta Methods
In convinient, the implementation of the function will take $[a_{ij}]$ as a numpy matrix while $b_i$ and $c_i$ will be 1-D numpy array known as parameter _rk_mat_, _weights_, _nodes_ respectively

In [23]:
def RK(f:Callable[[Union[float, int], Union[float, int]], Union[float, int]],
       a:Union[float, int],
       b:Union[float, int],
       rk_mat:np.ndarray[Union[float, int], Any],
       weights:np.ndarray[Union[float, int], Any],
       nodes:np.ndarray[Union[float, int], Any],
       y_init:Union[float, int],
       h:Union[float, int]=None,
       N:int=None,
        ) -> Tuple[np.ndarray[Union[float, int], Any], np.ndarray[Union[float, int], Any]]:
    if (h is None) and (N is None):
        raise ValueError("h and N cannot be None at the same time")
    elif h is None:
        h = (b - a) / N
    elif N is None:
        N = int((b - a) / h)
    if rk_mat.shape[0] != nodes.shape[0]:
        raise ValueError("The number of row of rk_mat not equal to the size of nodes...")
    if rk_mat.shape[1] != weights.shape[0]:
        raise ValueError("The number of column of rk_mat not equal to the size of weights...")
    
    t_, w_ = np.linspace(a, b, N + 1), np.ones((N + 1, )) * y_init
    for idx in range(1, N + 1):
        k_ = np.zeros((weights.shape[0], ), dtype=float)
        k_[0] = f(t_[idx - 1], w_[idx - 1])
        for jdx in range(1, weights.shape[0]):
            k_[jdx] += f(t_[idx - 1] + nodes[jdx - 1] * h, w_[idx - 1] + (rk_mat[jdx - 1] @ k_) * h)
        w_[idx] =  w_[idx - 1] + h * np.sum(k_ * weights)
    return (t_, w_)
    

In [24]:
import pandas as pd
f = lambda t, y: y - t ** 2 + 1

t_1, w_1 = RK2_Generic(f, 0, 2, 0.5, 0.2, alpha=0.5)
t_2, w_2 = RK2_Generic(f, 0, 2, 0.5, 0.2, alpha=1)
t_3, w_3 = RK2_Generic(f, 0, 2, 0.5, 0.2, alpha=2 / 3)
t_4, w_4 = Heun(f, 0, 2, 0.5, 0.2)
t_5, w_5 = RK4(f, 0, 2, 0.5, 0.2)


midpoint_rk = np.array([[0.5, 0]])
midpoint_c = np.array([0.5])
midpoint_b = np.array([0, 1])

modified_euler_rk = np.array([[1, 0]])
modified_euler_c = np.array([1])
modified_euler_b = np.array([0.5, 0.5])

ralston_rk = np.array([[2 / 3, 0]])
ralston_c = np.array([2 / 3])
ralston_b = np.array([1 / 4, 3 / 4])

heun3_rk = np.array([[1 / 3, 0, 0], [0, 2 / 3, 0]])
heun3_c = np.array([1 / 3, 2 / 3])
heun3_b = np.array([1 / 4, 0, 3 / 4])

RK4_rk = np.array([[1 / 2, 0, 0, 0], [0, 1 / 2, 0, 0], [0, 0, 1, 0]])
RK4_c = np.array([1 / 2, 1 / 2, 1])
RK4_b = np.array([1 / 6, 1 / 3, 1 / 3, 1 / 6])

t_6, w_6 = RK(f, 0, 2, midpoint_rk, midpoint_b, midpoint_c, 0.5, 0.2)
t_7, w_7 = RK(f, 0, 2, modified_euler_rk, modified_euler_b, modified_euler_c, 0.5, 0.2)
t_8, w_8 = RK(f, 0, 2, ralston_rk, ralston_b, ralston_c, 0.5, 0.2)
t_9, w_9 = RK(f, 0, 2, heun3_rk, heun3_b, heun3_c, 0.5, 0.2)
t_10, w_10 = RK(f, 0, 2, RK4_rk, RK4_b, RK4_c, 0.5, 0.2)


# The methods with _ at the end means that the values computed using
# the general formula
pd.DataFrame(
    {
        "t":t_1,
        "Midpoint":w_1,
        "Midpoint_":w_5,
        "Modified Euler":w_2,
        "Modified Euler_":w_6,
        "Ralston":w_3,
        "Ralston_":w_7,
        "Heun":w_4,
        "Heun_":w_8,
        "RK4":w_5,
        "RK4_":w_10
    }
)


,t,Midpoint,Midpoint_,Modified Euler,Modified Euler_,Ralston,Ralston_,Heun,Heun_,RK4,RK4_
0,0.0,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
1,0.2,0.828000,0.829293,0.826000,0.828000,0.827333,0.826000,0.829244,0.827333,0.829293,0.829293
2,0.4,1.211360,1.214076,1.206920,1.211360,1.209880,1.206920,1.213975,1.209880,1.214076,1.214076
3,0.6,1.644659,1.648922,1.637242,1.644659,1.642187,1.637242,1.648766,1.642187,1.648922,1.648922
4,0.8,2.121284,2.127203,2.110236,2.121284,2.117601,2.110236,2.126991,2.117601,2.127203,2.127203
5,1.0,2.633167,2.640823,2.617688,2.633167,2.628007,2.617688,2.640556,2.628007,2.640823,2.640823
6,1.2,3.170463,3.179894,3.149579,3.170463,3.163502,3.149579,3.179576,3.163502,3.179894,3.179894
7,1.4,3.721165,3.732340,3.693686,3.721165,3.712006,3.693686,3.731980,3.712006,3.732340,3.732340
8,1.6,4.270622,4.283409,4.235097,4.270622,4.258780,4.235097,4.283023,4.258780,4.283409,4.283409
9,1.8,4.800959,4.815086,4.755619,4.800959,4.785845,4.755619,4.814697,4.785845,4.815086,4.815086


## Chapter 5.5 Error Control and the Runge-Kutta-Fehlberg Method

In [73]:
def RKEmbedded(f:Callable[[Union[float, int], Union[float, int]], Union[float, int]],
            a:Union[float, int],
            b:Union[float, int],
            rk_mat:np.ndarray[Union[float, int], Any],
            weights:np.ndarray[Union[float, int], Any],
            weights_:np.ndarray[Union[float, int], Any],
            nodes:np.ndarray[Union[float, int], Any],
            y_init:Union[float, int],
            h_max:Union[float, int],
            h_min:Union[float, int],
            n:int,
            tol:float=1e-6,
        ) -> Tuple[np.ndarray[Union[float, int], Any], np.ndarray[Union[float, int], Any]]:

    if rk_mat.shape[0] != nodes.shape[0]:
        raise ValueError("The number of row of rk_mat not equal to the size of nodes...")
    if rk_mat.shape[1] != weights.shape[0]:
        raise ValueError("The number of column of rk_mat not equal to the size of weights...")
    
    h = h_max
    t_, w_ = [a], [y_init]
    idx = 1
    while True:
        k_ = np.zeros((weights.shape[0], ), dtype=float)
        k_[0] = f(t_[idx - 1], w_[idx - 1])
        for jdx in range(1, weights.shape[0]):
            k_[jdx] += f(t_[idx - 1] + nodes[jdx - 1] * h, w_[idx - 1] + (rk_mat[jdx - 1] @ k_) * h)
        R = abs((h * np.sum(k_ * weights)) - (h * np.sum(k_ * weights_))) / h
        if R <= tol:
            t_.append(t_[idx - 1] + h)
            w_.append(w_[idx - 1] + h * np.sum(k_ * weights))
        delta = 0.84 * (tol / R) ** (1 / n)
        if delta <= 0.1:
            delta = 0.1
        elif delta >= 4:
            delta = 4
        h *= delta
        if h > h_max:
            h = h_max
        if t_[idx] >= b:
            return (np.array(t_), np.array(w_))
        elif t_[idx] + h > b:
            h = b - t_[idx]
        elif h < h_min:
            return (np.array(t_), np.array(w_))
        idx += 1
    

In [74]:
RKF_rk = np.array([
    [0.25, 0, 0, 0, 0, 0], [3 / 32, 9 / 32, 0, 0, 0, 0],
    [1932 / 2197, -7200 / 2197, 7296 / 2197, 0, 0, 0],
    [439 / 216, -8, 3680 / 513, -845 / 4104, 0, 0],
    [-8 / 27, 2, -3544 / 2565, 1859 / 4104, -11 / 40, 0]
    ])
RKF_c = np.array([1 / 4, 3 / 8, 12 / 13, 1, 1 / 2])
RKF_bp = np.array([16 / 135, 0, 6656 / 12825, 28561 / 56430, -9 / 50, 2 / 55])
RKF_b = np.array([25 / 216, 0, 1408 / 2565, 2197 / 4104, -1 / 5, 0])

f = lambda t, y: y - t ** 2 + 1
RKEmbedded(f, 0, 2, RKF_rk, RKF_b, RKF_bp, RKF_c, 0.5, 0.25, 0.01, 4, 0.00001)

(array([0.       , 0.25     , 0.4865522, 0.7293332, 0.9793332, 1.2293332,
        1.4793332, 1.7293332, 1.9793332, 2.       ]),
 array([0.5       , 0.9204886 , 1.39649101, 1.95374879, 2.58642601,
        3.26046051, 3.95209554, 4.63082682, 5.25748606, 5.30548963]))

In [ ]:
# Exercise 1


## Chapter 5.6 Multistep Methods

In [ ]:
def BackwardDiffCoef(k:int):
    Q = np.zeros((k, k))
    Q[0, 0] = 1
    for idx in range(k - 1):
        for jdx in range(k - 1):
            if Q[idx, jdx] != 0:
                Q[idx, jdx + 1] += 1 * Q[idx, jdx]
                Q[idx + 1, jdx + 1] -= 1 * Q[idx, jdx]
    return Q

def ForwardDiffCoef(k:int):
    Q = np.zeros((k, k))
    Q[0, 0] = 1
    for idx in range(k - 1):
        for jdx in range(k - 1):
            if Q[idx, jdx] != 0:
                Q[idx, jdx + 1] -= 1 * Q[idx, jdx]
                Q[idx + 1, jdx + 1] += 1 * Q[idx, jdx]
    return Q

def CombCoef(m:int, backward:bool=True) -> np.ndarray:
    coef = np.ones((m, ))
    for mdx in range(1, m):
        roots = np.arange(mdx)
        poly = np.polynomial.Polynomial.fromroots(-roots if backward else roots)
        anti_poly = poly.integ()
        coef[mdx] = (anti_poly(1) - anti_poly(0)) / math.factorial(mdx)
    return coef


In [ ]:
def AdamsBashforth(f:Callable[[Union[float, int], Union[float, int]], Union[float, int]],
       a:Union[float, int],
       b:Union[float, int],
       y_init:Union[float, int],
       h:Union[float, int]=None,
       m:int=None,
       N:int=None) -> float:
    
    if (h is None) and (N is None):
        raise ValueError("h and N cannot be None at the same time")
    elif h is None:
        h = (b - a) / N
    elif N is None:
        N = int((b - a) / h)

    comb_coef = CombCoef(m)
    backward_coef = BackwardDiffCoef(m)

    coef = (backward_coef * comb_coef).sum(axis=1)
    t_, w_ = np.linspace(a, b, N + 1), np.ones((N + 1, ))
    w_[:m] = RK4(f, a, b, y_init, h)[1][:m]
    
    for idx in range(m, N + 1):
        f_val = np.array([f(t_[idx - jdx], w_[idx - jdx]) for jdx in range(1, m + 1)])
        w_[idx] = w_[idx - 1] + h * (f_val * coef).sum()
    return (t_, w_)


In [ ]:
f = lambda t, y: y - t ** 2 + 1
AdamsBashforth(f, 0, 2, 0.5, 0.2, 4)

(array([0. , 0.2, 0.4, 0.6, 0.8, 1. , 1.2, 1.4, 1.6, 1.8, 2. ]),
 array([0.5       , 0.82929333, 1.21407621, 1.64892202, 2.12728925,
        2.64105333, 3.18031413, 3.73301859, 4.28444241, 4.81659556,
        5.30750818]))

# 

In [ ]:
def AdamsMoulton(f:Callable[[Union[float, int], Union[float, int]], Union[float, int]],
                 f_:np.ndarray,
                 t:float,
                 w:float,
                 w_estimated:float,
                 h:float,
                 m:int 
                 ) -> float:
    '''
    For internal use of Predictor-Corrector Method only
    '''
    comb_coef = CombCoef(m, False)
    forward_coef = ForwardDiffCoef(m)
    coef = (forward_coef * comb_coef).sum(axis=1)
    f_cur = f(t, w_estimated) * coef[0]
    return w + h * (f_cur + (f_ * coef[1:]).sum())

def PredictorCorrector(f:Callable[[Union[float, int], Union[float, int]], Union[float, int]],
       a:Union[float, int],
       b:Union[float, int],
       y_init:Union[float, int],
       h:Union[float, int]=None,
       m:int=None,
       N:int=None) -> float:
    if (h is None) and (N is None):
        raise ValueError("h and N cannot be None at the same time")
    elif h is None:
        h = (b - a) / N
    elif N is None:
        N = int((b - a) / h)

    comb_coef = CombCoef(m)    
    backward_coef = BackwardDiffCoef(m)
    coef = (backward_coef * comb_coef).sum(axis=1)
    t_, w_ = np.linspace(a, b, N + 1), np.ones((N + 1, ))
    w_[:m] = RK4(f, a, b, y_init, h)[1][:m]
    

    for idx in range(m, N + 1):
        f_val = np.array([f(t_[idx - jdx], w_[idx - jdx]) for jdx in range(1, m + 1)])
        w_estimated = w_[idx - 1] + h * (f_val * coef).sum()
        w_[idx] = AdamsMoulton(f, f_val[:m - 1], t_[idx], w_[idx - 1], w_estimated, h, m)
    return (t_, w_)


In [ ]:
f = lambda t, y: y - t ** 2 + 1
PredictorCorrector(f, 0, 2, 0.5, 0.2, 4)

(array([0. , 0.2, 0.4, 0.6, 0.8, 1. , 1.2, 1.4, 1.6, 1.8, 2. ]),
 array([0.5       , 0.82929333, 1.21407621, 1.64892202, 2.12720563,
        2.6408286 , 3.17990264, 3.73235048, 4.28342082, 4.81509636,
        5.30537067]))